In [51]:
import numpy as np

# Getting Data

In [52]:
def find_horizon(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            if not line.startswith("#") and not line.startswith("SECTION_HORIZON"):
                n_days = int(line)
                break
    n_weekends = int(n_days/7)
    days = np.arange(1, n_days+1, dtype = int)
    weekends = np.arange(1, n_weekends+1, dtype = int)

    return n_days, days, n_weekends, weekends


In [61]:
def find_nurseIDs(file_path):
    NurseIDs = []
    inside_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_STAFF"):
                inside_block = True
            elif line.startswith("SECTION_DAYS_OFF"):
                inside_block = False
            elif inside_block:
                if line.split(',')[0] not in ['# ID', '']:
                    NurseIDs.append(line.split(',')[0])
    
    return NurseIDs

def find_shiftInfo(file_path):
    ShiftIDs = []
    Shift_lengths = []
    Forbidden_shifts = []
    inside_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_SHIFTS"):
                inside_block = True
            elif line.startswith("SECTION_STAFF"):
                inside_block = False
            elif inside_block:
                entries = line.split(',')
                if entries[0] not in ['# ShiftID', '']:
                    ShiftIDs.append(entries[0])
                    Shift_lengths.append(entries[1])
                    Forbidden_shifts.append(entries[2].split("|"))
    
    ForbiddenShifts = []
    for i, shift_list in enumerate(Forbidden_shifts):
        if '' in shift_list: shift_list.remove('')
        while len(shift_list) < len(ShiftIDs):
            shift_list.append('NA')
        ForbiddenShifts.append(shift_list)
            


    return ShiftIDs, np.array(Shift_lengths, dtype=int), ForbiddenShifts


In [62]:
def find_MaxShifts(file_path):
    num_shifts = len(find_shiftInfo(file_path)[0])
    MaxShifts = []
    inside_maxshift_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_STAFF"):
                inside_maxshift_block = True
            elif line.startswith("SECTION_DAYS_OFF"):
                inside_maxshift_block = False
            elif inside_maxshift_block:
                if line.split(',')[0] not in ['# ID', '']:
                    max_shifts = []
                    maxshift = line.split(',')[1]
                    entries = maxshift.split('|')
                    for shift in entries:
                        max_val = shift.split('=')[1]
                        max_shifts.append(int(max_val))
                    MaxShifts.append(max_shifts)
    
    return MaxShifts


def find_MaxMinInfo(file_path):
    MaxTotalMinutes = []
    MinTotalMinutes = []
    MaxConsecutiveShifts = []
    MinConsecutiveShifts = []
    MinConsecutiveDaysOff = []
    MaxWeekends = []
    inside_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_STAFF"):
                inside_block = True
            elif line.startswith("SECTION_DAYS_OFF"):
                inside_block = False
            elif inside_block:
                if line.split(',')[0] not in ['# ID', '']:
                    entries = line.split(',')
                    MaxTotalMinutes.append(entries[2])
                    MinTotalMinutes.append(entries[3])
                    MaxConsecutiveShifts.append(entries[4])
                    MinConsecutiveShifts.append(entries[5])
                    MinConsecutiveDaysOff.append(entries[6])
                    MaxWeekends.append(entries[7])
    
    return [MaxTotalMinutes, MinTotalMinutes,
    MaxConsecutiveShifts, MinConsecutiveShifts,
    MinConsecutiveDaysOff, MaxWeekends]


In [63]:
def find_DaysOff(file_path):
    DaysOff = []
    inside_block = False
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_DAYS_OFF"):
                inside_block = True
            elif line.startswith("SECTION_SHIFT_ON_REQUESTS"):
                inside_block = False
            elif inside_block:
                if line.split(',')[0] not in ['# EmployeeID', '']:
                    entries = line.split(',')
                    DaysOff.append(np.array(entries[1:], dtype=int)+1)
    return np.array(DaysOff)


In [64]:
def find_NotAssignedPreferredShiftPenalty(file_path):
    # Initialize variables
    shift_on_lines_to_save = []
    inside_on_block = False
    NurseIDs = find_nurseIDs(file_path)
    ShiftIDs = find_shiftInfo(file_path)[0]
    ndays = find_horizon(file_path)[0]
    Shift_On_Requests = []

    # Open the file for reading
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_SHIFT_ON_REQUESTS"):
                inside_on_block = True
            elif line.startswith("SECTION_SHIFT_OFF_REQUESTS"):
                inside_on_block = False
            elif inside_on_block:
                shift_on_lines_to_save.append(line)
    
    for i in NurseIDs:
        matrix = np.zeros((ndays, len(ShiftIDs)), dtype = int)
        for line in shift_on_lines_to_save[1:-1]:
            x = line.split(',')
            if x[0] == i:
                matrix[int(x[1]), int(ShiftIDs.index(x[2]))] = x[3]
        Shift_On_Requests.append(matrix.tolist())

    return np.array(Shift_On_Requests, dtype = int)



def find_AssignedNotPreferredShiftPenalty(file_path):
    # Initialize variables
    shift_off_lines_to_save = []
    inside_off_block = False
    NurseIDs = find_nurseIDs(file_path)
    ShiftIDs = find_shiftInfo(file_path)[0]
    ndays = find_horizon(file_path)[0]
    Shift_Off_Requests = []
    
    # Open the file for reading
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_SHIFT_OFF_REQUESTS"):
                inside_off_block = True
            elif line.startswith("SECTION_COVER"):
                inside_off_block = False
            elif inside_off_block:
                shift_off_lines_to_save.append(line)
    
    for i in NurseIDs:
        matrix = np.zeros((ndays, len(ShiftIDs)), dtype = int)
        for line in shift_off_lines_to_save[1:-1]:
            x = line.split(',')
            if x[0] == i:
                matrix[int(x[1]), int(ShiftIDs.index(x[2]))] = x[3]
        Shift_Off_Requests.append(matrix.tolist())

    return np.array(Shift_Off_Requests, dtype = int)

In [65]:
def find_CoverageRequirements(file_path):
    ShiftIDs = find_shiftInfo(file_path)[0]
    ndays, days = find_horizon(file_path)[:2]
    inside_block = False
    PreferredCoverage = np.zeros((ndays, len(ShiftIDs)), dtype = int)
    UnderstaffedPenalty = np.zeros((ndays, len(ShiftIDs)), dtype = int)
    OverstaffedPenalty = np.zeros((ndays, len(ShiftIDs)), dtype = int)

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespaces
            if line.startswith("SECTION_COVER"):
                inside_block = True
            # elif line.startswith("SECTION_SHIFT_OFF_REQUESTS"):
            #     inside_on_block = False
            elif inside_block:
                if line.split(',')[0] not in ["# Day", ""]:
                    entries = line.split(',')
                    row = int(entries[0])
                    col = int(ShiftIDs.index(entries[1]))
                    PreferredCoverage[row, col] = entries[2]
                    UnderstaffedPenalty[row, col] = entries[3]
                    OverstaffedPenalty[row,col] = entries[4]
    
    return PreferredCoverage, UnderstaffedPenalty, OverstaffedPenalty
            

    

In [66]:
instance_num = input("Choose a file number (between 1 and 24): ")
file_path = f'Instance_txts/Instance{instance_num}.txt'
print("You chose file: ", file_path)

You chose file:  Instance_txts/Instance1.txt


In [67]:
NurseIDs = find_nurseIDs(file_path)
ShiftIDs, Shift_lengths, Forbidden_shifts = find_shiftInfo(file_path)
MaxShifts = find_MaxShifts(file_path)
ndays, days, nweekends, weekends = find_horizon(file_path)
DaysOff = find_DaysOff(file_path)
NotAssignedPreferredShiftPenalty = find_NotAssignedPreferredShiftPenalty(file_path)
AssignedNotPreferredShiftPenalty = find_AssignedNotPreferredShiftPenalty(file_path)
PreferredCoverage, UnderstaffedPenalty, OverstaffedPenalty = find_CoverageRequirements(file_path)
MaxTotalMinutes = find_MaxMinInfo(file_path)[0]
MinTotalMinutes = find_MaxMinInfo(file_path)[1]
MaxConsecutiveShifts = find_MaxMinInfo(file_path)[2]
MinConsecutiveShifts = find_MaxMinInfo(file_path)[3]
MinConsecutiveDaysOff = find_MaxMinInfo(file_path)[4]
MaxWeekends = find_MaxMinInfo(file_path)[5]


# Writing the file

#### Format:
1. NurseID: set of strings


2. PlanningHorizon: integer
3. Days: set of integers

4. NumWeekends: integer
5. Weekends: set of integer

6. ShiftID: set of strings
7. ShiftLength: set of integer
8. ForbiddenShifts: set of strings

9. DaysOff: set of set of integers

10. MaxShifts: set of integers
 
11. MaxTotalMinutes:
12. MinTotalMinutes:
13. MaxConsecutiveShifts:
14. MinConsecutiveShifts:
15. MinConsecutiveDaysOff:
16. MaxWeekends:

17. PreferredCoverage: 

18. UnderstaffedPenalty:
 
19. OverstaffedPenalty:

20. AssignedNotPreferredShiftPenalty: 
 
21. NotAssignedPreferredShiftPenalty:


In [68]:
# Strings to be written before each array
# File path
output_file = f"Instance_dats/Instance{instance_num}.dat"

# Writing to the file
with open(output_file, 'w') as file:
    # Write NurseIDs
    file.write("NurseID: [")
    np.savetxt(file, NurseIDs, fmt='%s', delimiter=' ', newline=" ")  # You can adjust fmt and delimiter as needed
    file.write("] \n \n")

    # Write Horizon info
    file.write(f"PlanningHorizon: {ndays}")
    file.write("\n")
    file.write("Days: [")
    np.savetxt(file, days, fmt='%d', delimiter=' ', newline=" ")
    file.write("] \n \n")

    file.write(f"NumWeekends: {nweekends}")
    file.write("\n")
    file.write("Weekends: [")
    np.savetxt(file, weekends, fmt='%d', delimiter=' ', newline=" ")
    file.write("] \n \n")

    file.write("\n")

    # Write Shifts info
    file.write("ShiftID: [")
    np.savetxt(file, ShiftIDs, fmt='%s', delimiter=' ', newline=" ")  # You can adjust fmt and delimiter as needed
    file.write("] \n")
    file.write("ShiftLength: [")
    np.savetxt(file, Shift_lengths, fmt='%d', delimiter=' ', newline=" ")
    file.write("] \n")
    file.write("ForbiddenShifts: \n[ \n")
    np.savetxt(file, Forbidden_shifts, fmt='%s', delimiter=' ', newline="\n")  # You can adjust fmt and delimiter as needed
    file.write("] \n")

    file.write("\n")

    # Write days off info
    file.write("DaysOff: \n[")
    for row in DaysOff:
        file.write("[")
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline=']\n')
    file.write("] \n")

    file.write("\n")

    # Write MaxShifts info
    file.write("MaxShifts: \n[")
    for row in MaxShifts:
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write MaxMinInfo
    file.write("MaxTotalMinutes: [")
    np.savetxt(file, MaxTotalMinutes, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MinTotalMinutes: [")
    np.savetxt(file, MinTotalMinutes, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MaxConsecutiveShifts: [")
    np.savetxt(file, MaxConsecutiveShifts, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MinConsecutiveShifts: [")
    np.savetxt(file, MinConsecutiveShifts, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MinConsecutiveDaysOff: [")
    np.savetxt(file, MinConsecutiveDaysOff, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")
    file.write("MaxWeekends: [")
    np.savetxt(file, MaxWeekends, fmt='%s', delimiter=' ', newline=" ")  
    file.write("] \n")

    file.write("\n")

    # Write Preferred Coverage info
    file.write("PreferredCoverage: \n[")
    for row in PreferredCoverage:
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write UnderstaffedPenalty
    file.write("UnderstaffedPenalty: \n[")
    for row in UnderstaffedPenalty:
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write UnderstaffedPenalty
    file.write("OverstaffedPenalty: \n[")
    for row in OverstaffedPenalty:
        np.savetxt(file, [row], fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write AssignedNotPreferredPenalty
    file.write("AssignedNotPreferredShiftPenalty: \n[")
    for row in AssignedNotPreferredShiftPenalty:
            np.savetxt(file, row, fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")

    file.write("\n")

    # Write NotAssignedPreferredPenalty
    file.write("NotAssignedPreferredShiftPenalty: \n[")
    for row in NotAssignedPreferredShiftPenalty:
            np.savetxt(file, row, fmt='%d', delimiter=' ', newline='\n')
    file.write("] \n")


print(f"Arrays saved to {output_file}")

Arrays saved to Instance_dats/Instance1.dat
